In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import shutil


In [6]:
# Load the CSV file into a DataFrame
df = pd.read_csv('../data/August24_CB01/DataLog.txt', delimiter=',', names=['time', 'state', 'V'])

# Save the original Unix time in a new column
df['unix_time'] = df['time']

df.head()

,time,state,V,unix_time
0,1722872887,NoFlood,8176,1722872887
1,1722873247,NoFlood,8176,1722873247
2,1722873607,NoFlood,8176,1722873607
3,1722873967,NoFlood,8176,1722873967
4,1722874327,NoFlood,8176,1722874327


In [7]:
# Define the base directory where the images are stored
base_dir = '../data/August24_CB01/'

# Define the directory to save the non-dark images
output_dir = '../data/August24_CB01/non_dark_images/'
os.makedirs(output_dir, exist_ok=True)

In [8]:
# ok, lets see if we can remove the dark nightime images...

# Set the threshold for detecting dark images
brightness_threshold = 10  # Adjust this value as needed

# Initialize a list to store whether each image is too dark
too_dark = []

# Loop through the DataFrame
for index, row in df.iterrows():
    
    # Construct the full file path
    im_path = str(row['unix_time']) + row['state'] + '.jpg'
    full_path = os.path.join(base_dir, im_path)
    
    try:
        # Load the image and convert to grayscale
        image = Image.open(full_path).convert('L')
        
        # Convert the image to a NumPy array
        image_array = np.array(image)
        
        # Calculate the average brightness
        avg_brightness = np.mean(image_array)
        
        # Determine if the image is too dark
        if avg_brightness > brightness_threshold:
            too_dark.append(False)  # Image is not too dark
            
            # Copy the image to the new folder
            shutil.copy(full_path, os.path.join(output_dir, im_path))
        else:
            too_dark.append(True)   # Image is too dark
    except Exception as e:
        print(f"Warning: Could not process image at {full_path}. Error: {e}")
        too_dark.append(None)  # Mark as None if the image couldn't be processed

# Add the results to a new column in the DataFrame
df['too_dark'] = too_dark

# Display the first few rows of the updated DataFrame
df.head()


,time,state,V,unix_time,too_dark
0,1722872887,NoFlood,8176,1722872887,False
1,1722873247,NoFlood,8176,1722873247,False
2,1722873607,NoFlood,8176,1722873607,False
3,1722873967,NoFlood,8176,1722873967,False
4,1722874327,NoFlood,8176,1722874327,False
